<font size=6 color=red>ACCによる定速走行データ抽出プログラム</font>

In [ ]:
import os
import re
from pathlib import Path
from os import path
import json
import time
import pyodbc
import webbrowser
import shutil
import csv
import pandas as pd
#import folium
import datetime
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import pyodbc
import numpy as np
import math
import datetime
import warnings
warnings.simplefilter('ignore')

## ACC判定関数

***
DBから取ってきたECOLOGデータのうち、以下の条件を満たすtripを取得
***

定速判定の条件（tripごとに判定する）
$$
\frac{\sum(v_i - AccSettingSpeed \times 0.92)^2}{N}\\
$$
(i:レコード番号)
(N:レコード数)

In [ ]:
def is_acc(acc_setting_speed,df):
    
    var_like_list=[]
    trip_list= list(set(df.Trip_id.to_list()))
    for trip in trip_list :
        df_trip = df[df.Trip_id.isin([trip])]
        var_like=0
        for i in range(len(df_trip)):
            var_like=var_like + (df_trip.speed.iloc[i] - acc_setting_speed*0.92)**2
            
        var_like=var_like/len(df_trip)
        
        if var_like<12:            #16 is best 
            var_like_list.append(trip)
    return var_like_list

## DBと接続
***
get_datas_from_tables()：
DBから指定のSemanticLinkの走行ログを取得
***
***
get_leafspy_datas_from_tables():
LeafSpyのデータを取得
***

In [ ]:
def get_datas_from_tables(start_date, car_id,semanticlink_id):
    driver='{SQL Server}'
    server = 'ecologdatabasesystem2030'
    database = 'ECOLOGDdatabaseppp2030'
    trusted_connection='yes'
    end_date=datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')

    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    
    sql = """WITH S AS (
                  SELECT
                    SL.LINK_ID
                    ,SL.SEMANTIC_LINK_ID
                  FROM
                    SEMANTIC_LINKS AS SL
                  WHERE
                    SL.SEMANTIC_LINK_ID = {}

                ), ES AS (
                  SELECT
                    TRIP_ID
                    ,E.LATITUDE
                    ,E.LONGITUDE
                    ,S.LINK_ID
                    ,E.JST
                    ,SPEED
                    ,SENSOR_ID
                    ,CAR_ID
                  FROM
                    ECOLOG_Doppler_NotMM AS E INNER JOIN S 
                      ON E.LINK_ID = S.LINK_ID
                  WHERE SENSOR_ID in (31,34)
                )

                SELECT 
                  TRIP_ID
                  ,LATITUDE
                  ,LONGITUDE
                  ,LINK_ID
                  ,JST
                  ,SPEED
                 FROM ES
                 
                  WHERE
                    JST BETWEEN '{}' AND '{}'
                    AND CAR_ID = {}
                  ORDER BY JST
            """.format(str(semanticlink_id),start_date,end_date, str(car_id))

    cursor.execute(sql) 
    rows = cursor.fetchall()
    df_rows = pd.DataFrame({'Trip_id':[],
                            'Lat':[],
                            'Lon':[],
                            'Link_id':[],
                            'JST':[],
                            'speed':[]})
    
        
    df_rows.loc[:]= tuple(rows)
    
    cursor.close()
    connect.close()
    return df_rows


def get_leafspy_datas_from_LEAFSPY_RAW_PROJECTED_ALL(start_date, end_date,AC_POWER_250W_condition):
    driver='{SQL Server}'
    server = 'ecologdatabasesystem2030'
    database = 'ECOLOGDdatabaseppp2030'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    
    if AC_POWER_250W_condition>0:
        sql = """SELECT JST,
             DATEDIFF(SECOND, LAG(L.JST, 1) OVER (ORDER BY L.JST), L.JST) AS SECONDS_DIFF
            ,L.PACK_AMP * L.PACK_VOLTS / 3600 AS PACK_Wh
            ,L.AUX_POWER_100W * 100 AS AUX_POWER_W
            ,L.AC_POWER_250W * 250 AS AC_POWER_W
          FROM
            LEAFSPY_RAW_PROJECTED_ALL AS L
          WHERE L.AC_POWER_250W < {}
          AND L.AUX_POWER_100W < 300
          AND JST BETWEEN '{}' AND '{}'
          order by JST
            """.format(AC_POWER_250W_condition,start_date,end_date)
    else:
        sql = """SELECT JST,
                 DATEDIFF(SECOND, LAG(L.JST, 1) OVER (ORDER BY L.JST), L.JST) AS SECONDS_DIFF
                ,L.PACK_AMP * L.PACK_VOLTS / 3600 AS PACK_Wh
                ,L.AUX_POWER_100W * 100 AS AUX_POWER_W
                ,L.AC_POWER_250W * 250 AS AC_POWER_W
              FROM
                LEAFSPY_RAW_PROJECTED_ALL AS L
              WHERE L.AC_POWER_250W = 0
              AND L.AUX_POWER_100W < 300
              AND JST BETWEEN '{}' AND '{}'
              order by JST
                """.format(start_date,end_date)
    

    cursor.execute(sql) 
    rows = cursor.fetchall()
    df_rows = pd.DataFrame({'JST':[],
                            'SECONDS_DIFF':[],
                            'PACK_Wh':[],
                            'AUX_POWER_W':[],
                            'AC_POWER_W':[]})

    

    if len(rows)>0:
        df_rows.loc[:]= tuple(rows)
    cursor.close()
    connect.close()
    return df_rows

def get_leafspy_datas_from_LEAFSPY_RAW_PROJECTED(start_date, end_date,AC_POWER_250W_condition):
    driver='{SQL Server}'
    server = 'ecologdatabasesystem2030'
    database = 'ECOLOGDdatabaseppp2030'
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    
    if AC_POWER_250W_condition>0:
        sql = """SELECT JST,
             DATEDIFF(SECOND, LAG(L.JST, 1) OVER (ORDER BY L.JST), L.JST) AS SECONDS_DIFF
            ,L.PACK_AMP * L.PACK_VOLTS / 3600 AS PACK_Wh
            ,L.AUX_POWER_100W * 100 AS AUX_POWER_W
            ,L.AC_POWER_250W * 250 AS AC_POWER_W
          FROM
            LEAFSPY_RAW_PROJECTED AS L
          WHERE L.AC_POWER_250W < {}
          AND L.AUX_POWER_100W < 300
          AND JST BETWEEN '{}' AND '{}'
          order by JST
            """.format(AC_POWER_250W_condition,start_date,end_date)
    else:
        sql = """SELECT JST,
                 DATEDIFF(SECOND, LAG(L.JST, 1) OVER (ORDER BY L.JST), L.JST) AS SECONDS_DIFF
                ,L.PACK_AMP * L.PACK_VOLTS / 3600 AS PACK_Wh
                ,L.AUX_POWER_100W * 100 AS AUX_POWER_W
                ,L.AC_POWER_250W * 250 AS AC_POWER_W
              FROM
                LEAFSPY_RAW_PROJECTED AS L
              WHERE L.AC_POWER_250W = 0
              AND L.AUX_POWER_100W < 300
              AND JST BETWEEN '{}' AND '{}'
              order by JST
                """.format(start_date,end_date)
    

    cursor.execute(sql) 
    rows = cursor.fetchall()
    df_rows = pd.DataFrame({'JST':[],
                            'SECONDS_DIFF':[],
                            'PACK_Wh':[],
                            'AUX_POWER_W':[],
                            'AC_POWER_W':[]})
 
    if len(rows)>0:
        df_rows.loc[:]= tuple(rows)
    
    cursor.close()
    connect.close()
    return df_rows

In [ ]:
list_semantic_links = []


print('＊＊＊DBからACC定速走行データを取得します＊＊＊\n\n')

print('SemanticLinkIDを入力してください')
semantic_id = int(input())
print('ACCの設定速度を入力してください')
acc_setting_speed = int(input())
print('---------------')

speed_changing_list = [5,10,15,20]
AC_setting_changing_list = [0,2]
list_semantic_links.append([semantic_id,acc_setting_speed,0])
list_semantic_links.append([semantic_id,acc_setting_speed,2])

for i in range(len(speed_changing_list)):
    for j in range(len(AC_setting_changing_list)):
        list_semantic_links.append([semantic_id,acc_setting_speed+speed_changing_list[i],AC_setting_changing_list[j]])
        list_semantic_links.append([semantic_id,acc_setting_speed-speed_changing_list[i],AC_setting_changing_list[j]])


for i in range(len(list_semantic_links)):
    Speed = list_semantic_links[i][1]
    ACsetting = list_semantic_links[i][2]
    
    if ACsetting > 0:
        ACsetting_output = (ACsetting-1)*250
    else:
        ACsetting_output = 0
    # 出力ファイルのパス
    path= rf"{os.getcwd()}\output_csv\{semantic_id}_acc_setting_speed{Speed}_AC_setting{ACsetting_output}W.csv"
    # データベースからECOLOGデータを取得
    df_datas_from_tables = get_datas_from_tables('2020-08-01', 22,semantic_id)
    # 取得したECOLOGデータに定速判定をかける
    df_reslut = df_datas_from_tables[df_datas_from_tables.Trip_id.isin(is_acc(list_semantic_links[i][1],df_datas_from_tables))]
    # 処理の関係で、定速条件を満たしたtripの開始時間と終了時間を取り出してる
    Jst_min_list = df_reslut.groupby('Trip_id').JST.min().to_list()
    Jst_max_list = df_reslut.groupby('Trip_id').JST.max().to_list()
    df_leafspy = pd.DataFrame({'JST':[],
                            'SECONDS_DIFF':[],
                            'PACK_Wh':[],
                            'AUX_POWER_W':[],
                            'AC_POWER_W':[]})
    if len(Jst_min_list)>0:
        # trip_idごとに定速条件を満たしたLeafSpyのデータをとってくる
        for i in range(len(Jst_min_list)):
            df_not_PROJECTED_ALL = get_leafspy_datas_from_LEAFSPY_RAW_PROJECTED(str(Jst_min_list[i]),str(Jst_max_list[i]),ACsetting)
            df_PROJECTED_ALL = get_leafspy_datas_from_LEAFSPY_RAW_PROJECTED_ALL(str(Jst_min_list[i]),str(Jst_max_list[i]),ACsetting)
            if len(df_PROJECTED_ALL)>0:
                df_leafspy=pd.concat([df_leafspy,df_PROJECTED_ALL])
                if len(df_not_PROJECTED_ALL)>0:
                    if df_leafspy.JST.isin([df_not_PROJECTED_ALL.iloc[-1]]).any():
                        df_leafspy=pd.concat([df_leafspy,df_not_PROJECTED_ALL])
            else:
                if len(df_not_PROJECTED_ALL)>0:
                    if df_leafspy.JST.isin([df_not_PROJECTED_ALL.iloc[-1]]).any():
                        df_leafspy=pd.concat([df_leafspy,df_not_PROJECTED_ALL])
                        continue
                    else:
                        continue
                else:
                    continue
                
        # LeafSpyとECOLOGをJOIN
        #sometims df_leafspy.JST is not timestemp,is object
        #so,for merging them corectly,we convert df_reslut.JST to object,too
        df_reslut.JST = df_reslut.JST.astype(object) 
        df_output=pd.merge(df_leafspy, df_reslut, on='JST')
        # いらないカラムを捨てる
        df_output.drop(['Link_id'],axis = 1,inplace=True)
        # データフレームのカラムの順番入れ替え
        df_output =df_output[['JST','Trip_id','Lat','Lon','speed','PACK_Wh','AUX_POWER_W','AC_POWER_W','SECONDS_DIFF']]
        # 取得データをcsvとして出力
        df_output.to_csv(path)
        print('In Semantic Link {},condition: ACC setting speed {}km/h ,AC Power setting {}W'
              .format(semantic_id,Speed ,ACsetting_output))
        print('＊＊＊出力完了＊＊＊\n\n')
        print('---------------')
    else:
        print('In Semantic Link {},condition: ACC setting speed {}km/h ,AC Power setting {}W'
              .format(semantic_id,Speed ,ACsetting_output))
        print('＊＊＊There are not any data sets in your setting conditions＊＊＊')
        print('---------------')
        continue
        
print('All process are done')

# Cread

- Sogaと一緒にこのプログラムを完成した